In [1]:

# Built In Imports
import xml.etree.ElementTree as ET
from datetime import datetime
from glob import glob
import warnings
import IPython
import urllib
import zipfile
import pickle
import shutil
import string
import math
import tqdm
import time
import os
import gc
import re

# Visualization Imports
FIG_FONT = dict(family="Helvetica, Arial", size=14, color="#7f7f7f")
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from lxml import etree
import seaborn as sns
from PIL import Image
import plotly
import PIL
import cv2

# Other Imports
#from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.notebook import tqdm
#import pydicom

In [2]:
pd.options.mode.chained_assignment

NameError: name 'pd' is not defined

In [ ]:
pd.options.mode.chained_assignment = None

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="background_information">2&nbsp;&nbsp;BACKGROUND INFORMATION</a>

* Pascal VOC provides standardized image data sets for object detection. 
* Pascal VOC is an XML file, unlike COCO which has a JSON file.
* In Pascal VOC we create a file for each of the image in the dataset. 
* The bounding Box in Pascal VOC is formatted as: **(xmin, ymin, xmax, ymax)**

**Here is an example**
![Example of Pascal VOC Annotation](https://miro.medium.com/max/1130/1*J84PBv70HWVW_tJ2zQwd4g.png)

In [3]:
import pandas as pd

# Load the val.csv file
val_csv_path = '../data/Stream1/labels/val.csv'
val_df = pd.read_csv(val_csv_path)

# Display the first few rows of the dataframe
val_df.head()


,filename,class,bbox
0,img055517.png,lisa_pathfinder,"[375, 273, 610, 496]"
1,img056814.png,lisa_pathfinder,"[678, 302, 811, 448]"
2,img053102.png,lisa_pathfinder,"[246, 247, 466, 473]"
3,img053516.png,lisa_pathfinder,"[338, 600, 601, 878]"
4,img055023.png,lisa_pathfinder,"[545, 408, 762, 596]"


In [4]:
import pandas as pd

# Load the train.csv file
train_csv_path = '../data/Stream1/labels/train.csv'
train_df = pd.read_csv(train_csv_path)

# Display the first few rows of the dataframe
train_df.head()


,filename,class,bbox
0,img086351.png,smart_1,"[183, 311, 657, 415]"
1,img083734.png,smart_1,"[361, 352, 479, 716]"
2,img081317.png,smart_1,"[163, 16, 905, 785]"
3,img089140.png,smart_1,"[188, 427, 499, 573]"
4,img082379.png,smart_1,"[152, 462, 1024, 605]"


In [6]:
import pandas as pd

# Load the train.csv file
test_csv_path = '../data/Stream1/labels/test.csv'
test_df = pd.read_csv(test_csv_path)

# Display the first few rows of the dataframe
test_df.head()


,filename,class,bbox
0,img051616.png,lisa_pathfinder,"[123, 699, 427, 1024]"
1,img013814.png,cheops,"[411, 663, 670, 910]"
2,img059033.png,lisa_pathfinder,"[212, 623, 313, 733]"
3,img030819.png,double_star,"[27, 339, 1024, 736]"
4,img052131.png,lisa_pathfinder,"[423, 563, 645, 797]"


In [7]:
# Load the pascal_template.xml file
template_xml_path = './pascal_template.xml'

with open(template_xml_path, 'r') as file:
    template_xml = file.read()

print(template_xml)


<annotation>
	<folder></folder>
	<filename>{filename}</filename>
	<path>{filename}</path>
	<source>
		<database>Spark.Stream1</database>
	</source>
	<size>
		<width>1024</width>
		<height>1024</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>{class}</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<occluded>0</occluded>
		<bndbox>
			<ymin>{bbox[0]}</ymin>
			<xmin>{bbox[1]}</xmin>
			<ymax>{bbox[2]}</ymax>
			<xmax>{bbox[3]}</xmax>
		</bndbox>
	</object>
</annotation>



In [8]:
import os
import xml.etree.ElementTree as ET
from xml.dom import minidom

def create_xml_file(row, template, output_dir):
    """
    Create an XML file based on the template and the provided row data.

    :param row: A row from the dataframe containing filename, class, and bbox.
    :param template: XML template as a string.
    :param output_dir: Directory to save the XML files.
    """
    # Replace filename (with .jpg extension) in the template
    filename_jpg = row['filename'].replace('.png', '.jpg')
    xml_content = template.replace('{filename}', filename_jpg)

    # Replace class in the template
    xml_content = xml_content.replace('{class}', row['class'])

    # Replace bounding box values in the template
    bbox = eval(row['bbox'])  # Convert string representation of list to actual list
    for i in range(4):
        xml_content = xml_content.replace(f'{{bbox[{i}]}}', str(bbox[i]))

    # Parse the modified XML content
    xml_tree = ET.ElementTree(ET.fromstring(xml_content))

    # Create the XML file
    xml_filename = os.path.join(output_dir, row['filename'].replace('.png', '.xml'))
    xml_tree.write(xml_filename)

    return xml_filename



In [9]:
# Directory to save the XML files
output_dir = './val'
os.makedirs(output_dir, exist_ok=True)

# Generate XML files
xml_files = [create_xml_file(row, template_xml, output_dir) for index, row in val_df.iterrows()]

# Return the list of generated XML file paths
len(xml_files)


22000

In [10]:
# Directory to save the XML files
output_dir = './train'
os.makedirs(output_dir, exist_ok=True)

# Generate XML files
xml_files = [create_xml_file(row, template_xml, output_dir) for index, row in train_df.iterrows()]

# Return the list of generated XML file paths
len(xml_files)


66000

In [11]:
# Directory to save the XML files
output_dir = './test'
os.makedirs(output_dir, exist_ok=True)

# Generate XML files
xml_files = [create_xml_file(row, template_xml, output_dir) for index, row in test_df.iterrows()]

# Return the list of generated XML file paths
len(xml_files)


22000